In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
dropout_prob = tf.placeholder(tf.float32)

### Layers

#### Layer 1

In [4]:
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
# padding='SAME' keeps output size equal to input
L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)
# Pooling layer
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

#### Layer 2

In [5]:
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

#### Layer 3

In [6]:
W3 = tf.Variable(tf.random_normal([7 * 7 * 64, 256], stddev=0.01))
L3 = tf.reshape(L2, [-1, 7 * 7 * 64])
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)
# Dropout
L3 = tf.nn.dropout(L3, dropout_prob)

#### Layer 4

In [7]:
W4 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L3, W4)

In [8]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.005).minimize(cost)

In [9]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [10]:
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

In [12]:
for epoch in range(10):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)    # When image number is unknown put -1
        _, curr_loss = sess.run([optimizer, cost],
                                feed_dict={X: batch_xs, Y: batch_ys, dropout_prob: 0.7})
        total_cost += curr_loss
        
    print('Epoch: ', '%04d' % (epoch + 1))
    print('Avg cost: ', '{:,.3f}'.format(total_cost / total_batch))

Epoch:  0001
Avg cost:  0.064
Epoch:  0002
Avg cost:  0.046
Epoch:  0003
Avg cost:  0.037
Epoch:  0004
Avg cost:  0.030
Epoch:  0005
Avg cost:  0.029
Epoch:  0006
Avg cost:  0.026
Epoch:  0007
Avg cost:  0.025
Epoch:  0008
Avg cost:  0.021
Epoch:  0009
Avg cost:  0.025
Epoch:  0010
Avg cost:  0.019


In [13]:
correctness = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correctness, tf.float32))
print('Accuracy: ', sess.run(accuracy,
                            feed_dict={X: mnist.test.images.reshape(-1, 28, 28, 1),
                                      Y: mnist.test.labels, dropout_prob: 1}))    # No dropout

Accuracy:  0.9897
